# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# 作業1
* 參考範例，將鐵達尼的艙位代碼( 'Cabin' )欄位使用特徵雜湊 / 標籤編碼 / 目標均值編碼三種轉換後，  
與其他數值型欄位一起預估生存機率

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = '../data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
#分類類別值型欄位, 存於 object_features & num_features 中
object_features = []
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
    else :
        num_features.append(feature)
print(f'{len(object_features)} Object Features : {object_features}\n')
print(f'{len(num_features)} Num Features : {num_features}\n')


# 保留類別型欄位
dfobj = df[object_features]
dfobj = dfobj.fillna('None')
train_num = train_Y.shape[0]

### 補留數值欄位，以便後續使用
dfnum = df[num_features]
dfnum = dfnum.fillna(df.mean())

dfobj.head()

5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

5 Num Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


# 作業2
* 承上題，三者比較效果何者最好?

In [3]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in dfobj.columns:
    df_temp[c] = LabelEncoder().fit_transform(dfobj[c])
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.780004837244799


,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3


### DataFrame.aggregate(func, axis=0, *args, **kwargs) 
>Aggregate using one or more operations over the specified axis

In [4]:
# 加上 'Cabin' 欄位的計數編碼
count_df = dfobj.groupby(['Cabin'])['Name'].agg({'Cabin_Count':'size'}).reset_index()
df = pd.merge(dfobj, count_df, on=['Cabin'], how='left')
### 查看結果
df.head(10)

C:\Users\kevin\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Count
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S,1014
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C,2
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S,1014
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S,2
4,"Allen, Mr. William Henry",male,373450,None,S,1014
5,"Moran, Mr. James",male,330877,None,Q,1014
6,"McCarthy, Mr. Timothy J",male,17463,E46,S,2
7,"Palsson, Master. Gosta Leonard",male,349909,None,S,1014
8,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,347742,None,S,1014
9,"Nasser, Mrs. Nicholas (Adele Achem)",female,237736,None,C,1014


In [5]:
count_df.sort_values(by=['Cabin_Count'], ascending=False).head(10)

,Cabin,Cabin_Count
185,None,1014
80,C23 C25 C27,6
184,G6,5
47,B57 B59 B63 B66,5
60,B96 B98,4
180,F2,4
117,D,4
79,C22 C26,4
181,F33,4
183,F4,4


In [26]:
# 'Cabin'計數編碼 + 邏輯斯迴歸
df_temp2 = pd.DataFrame()
for c in dfobj.columns:
    df_temp2[c] = LabelEncoder().fit_transform(dfobj[c])
df_temp2['Cabin_Count'] = df['Cabin_Count']

###由於 concat 一直出錯，重新排列index###
df_temp2.reset_index(drop=True, inplace=True)
dfnum.reset_index(drop=True, inplace=True)
###############################
df_temp2 = pd.concat([df_temp2,dfnum],axis=1)

train_X = df_temp2[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())

0.7912664679991787


In [34]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp3 = pd.DataFrame()
for c in object_features:
    df_temp3[c] = LabelEncoder().fit_transform(dfobj[c])
df_temp3['Cabin_Hash'] = df['Cabin'].map(lambda x:hash(x) % 10)
df_temp3 = pd.concat([df_temp3, dfnum], axis=1)
train_X = df_temp3[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())

0.7890380372701908


In [36]:
# 'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp4 = pd.DataFrame()
for c in object_features:
    df_temp4[c] = LabelEncoder().fit_transform(dfobj[c])

df_temp4['Cabin_Count'] = df['Cabin_Count']
df_temp4['Cabin_Hash'] = df['Cabin'].map(lambda x:hash(x) % 10)

df_temp4 = pd.concat([df_temp4, dfnum], axis=1)
train_X = df_temp4[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X , train_Y, cv=5).mean())

0.7879018876248454
